In [2]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import matplotlib.pyplot as plt

In [5]:
url = 'https://github.com/LukeCaptainAI/norm_quad/blob/main/aisdecoded.csv?raw=true' # forked from github
local = '/home/luke/norm_quad/aisdecoded2.csv'
df = pd.read_csv(url, index_col=0)

# df['delta'] = df['true_heading'] - df['cog']
da = df['delta']
del df['delta']
df['delta'] = da.abs() 
df

KeyError: 'delta'

In [3]:
trig = np.sin(np.radians((df['delta']))*(-1)) # *-1: delta = hdn-cog, hdn > cog >>> starboard, hdn < cog >>> Portside 
d_offset= (df.sog*trig) # d_offset is the magnitude of the offtrack vector perpendicular to the heading.
df['d_offset'] = d_offset.tolist()


heading_adrift = np.select([(df.delta < 0),
                        (df.delta == 0),
                        (df.delta > 0)],
                        ['starboard','no drift', 'portside'])

df['drift_to'] = heading_adrift.tolist()

In [4]:
df['cog'].describe()

count   120.00
mean    137.45
std      98.94
min       1.00
25%      53.25
50%     111.45
75%     231.68
max     336.00
Name: cog, dtype: float64

In [5]:
pika = 'https://iitutor.com/wp-content/uploads/2019/03/N1303-Signs-in-Quadrants.png'
from IPython.display import Image # trigonomic functions of quadrants
Image(url= pika, width=400, height=400)

 
print('Rechts draaiend stelsel...')
print(' ')
print('Q4     0      Q1')
print('               ')
print('       |        ')
print('270  --|--    90')
print('       |        ')
print('               ')
print('Q3    180     Q2')

pika = 'https://iitutor.com/wp-content/uploads/2019/03/N1303-Signs-in-Quadrants.png'
from IPython.display import Image # trigonomic functions of quadrants
Image(url= pika, width=400, height=400)

Rechts draaiend stelsel...
 
Q4     0      Q1
               
       |        
270  --|--    90
       |        
               
Q3    180     Q2


In [6]:
cog = np.array(df['cog'])
hdn = np.array(df['true_heading'])


cog # geisoleerde numpy array voor indivudeele operaties

array([304.        , 115.90000153, 106.59999847,  43.        ,
        96.        , 260.6000061 , 111.        , 294.        ,
       103.90000153,  15.39999962, 106.59999847, 268.29998779,
        61.5       , 133.        ,  94.        , 304.6000061 ,
       116.80000305, 227.1000061 ,  42.        ,  42.        ,
       111.30000305,  90.        , 114.19999695,  35.        ,
       106.59999847, 298.29998779, 114.5       ,   9.30000019,
        85.        ,  41.        , 293.        , 111.59999847,
       113.40000153,  79.        , 120.80000305,  50.5       ,
       267.70001221, 112.09999847,  73.        , 156.3999939 ,
        15.19999981, 112.69999695, 291.        , 106.59999847,
        67.        , 305.20001221,  43.        , 111.69999695,
       290.        ,  41.        , 114.40000153, 112.        ,
       298.6000061 ,  27.        ,  60.        , 336.        ,
        54.        , 291.        ,  24.        ,  50.20000076,
        51.        , 111.09999847, 123.69999695, 106.59

In [7]:
# isolation of: 1.quadrant deterimation 2. reduced angle based on quadrant 3. magnitude vector * cos_x and sin_x

# cog 1
quad_cog = np.select([(df.cog>0)&(df.cog<90),
                     (df.cog>90)&(df.cog<180),
                      (df.cog>180)&(df.cog<270),
                     (df.cog>270)&(df.cog<360)],
                    [1,2,3,4])

df['qd_cog'] = quad_cog.tolist()

#df['red_hdn'] = red_hdn.tolist()
#df['qd_hdn'] = quad_hdn.tolist()

# 2
red_cog = np.select([(df.cog>0)&(df.cog<90),
                     (df.cog>90)&(df.cog<180),
                      (df.cog>180)&(df.cog<270),
                     (df.cog>270)&(df.cog<360)],
                    [df.cog,df.cog-90,df.cog-180,df.cog-270])

df['red_cog'] = red_cog.tolist()

# 3 cog x, y formula
a = (df.sog*np.cos(np.radians(df.red_cog)))
b = (df.sog*np.cos(np.radians(df.red_cog))*(-1))
c = (df.sog*np.sin(np.radians(df.red_cog)))
d = (df.sog*np.sin(np.radians(df.red_cog))*(-1))

cog_x =   np.select([(df.cog>0)&(df.cog<90),
                     (df.cog>90)&(df.cog<180),
                      (df.cog>180)&(df.cog<270),
                     (df.cog>270)&(df.cog<360)],
                    [a, a, b, b])

cog_y = np.select([(df.cog>0)&(df.cog<90),
                     (df.cog>90)&(df.cog<180),
                      (df.cog>180)&(df.cog<270),
                     (df.cog>270)&(df.cog<360)],
                    [c, d, d, c])

df['cog_x'] = cog_x.tolist()
df['cog_y'] = cog_y.tolist()

# hdn 1
quad_hdn = np.select([(df.true_heading>0)&(df.true_heading<90),
                     (df.true_heading>90)&(df.true_heading<180),
                      (df.true_heading>180)&(df.true_heading<270),
                     (df.true_heading>270)&(df.true_heading<360)],
                    [1,2,3,4])

# hdn 2
red_hdn = np.select([(df.true_heading>0)&(df.true_heading<90),
                     (df.true_heading>90)&(df.true_heading<180),
                      (df.true_heading>180)&(df.true_heading<270),
                     (df.true_heading>270)&(df.true_heading<360)],
                    [df.true_heading, df.true_heading-90, df.true_heading-180, df.true_heading-270])

In [8]:


#hdn x, y formula
#e = (df.sog*np.cos(np.radians(df.red_hdn)))
#f = (df.sog*np.cos(np.radians(df.red_hdn))*(-1))
#g = (df.sog*np.sin(np.radians(df.red_hdn)))
#h = (df.sog*np.sin(np.radians(df.red_hdn))*(-1))



cog_x =   np.select([(df.cog>0)&(df.cog<90),
                     (df.cog>90)&(df.cog<180),
                      (df.cog>180)&(df.cog<270),
                     (df.cog>270)&(df.cog<360)],
                    [a, a, b, b])
                  
cog_y = np.select([(df.cog>0)&(df.cog<90),
                     (df.cog>90)&(df.cog<180),
                      (df.cog>180)&(df.cog<270),
                     (df.cog>270)&(df.cog<360)],
                    [c, d, d, c])

# hdn_x = np.select([(df.true_heading>0)&(df.true_heading<90),
#                      (df.true_heading>90)&(df.true_heading<180),
#                       (df.true_heading>180)&(df.true_heading<270),
#                      (df.true_heading>270)&(df.true_heading<360)],
#                     [e, e, f, f])
# 
# hdn_y = np.select([(df.true_heading>0)&(df.true_heading<90),
#                      (df.true_heading>90)&(df.true_heading<180),
#                       (df.true_heading>180)&(df.true_heading<270),
#                      (df.true_heading>270)&(df.true_heading<360)],
#                     [g, h, h, g])

In [17]:
df['qd_cog'] = quad_cog.tolist()
df['red_cog'] = red_cog.tolist()
df['cog_x'] = cog_x.tolist()
df['cog_y'] = cog_y.tolist()
df['red_hdn'] = red_hdn.tolist()
df['qd_hdn'] = quad_hdn.tolist()

df[['utc','mmsi','sog','cog','qd_cog','red_cog','true_heading','qd_hdn','red_hdn','delta','d_offset','drift_to']]


KeyError: "['d_offset', 'utc', 'drift_to'] not in index"

In [25]:
# Use cosine rule to solve for drift magnitude method : B
s_own = 10.1 # O
sog = 11.4 # s
cog = 304 
hdn = 313 #
delta = 9  # d

# a^2 = b^2 + c^2 - 2 bc cosA
d_square = (sog**2 + s_own**2) - (2 * s_own * sog * np.cos(np.radians(delta)))
d = np.sqrt(d_square)
d


# Use cosine rule to solve for angle B in triangle abc
# cos B = (a^2 + b^2 - c^2) / 2 a c 

S = 

2.1272350429023272

In [10]:
sin = 'https://github.com/LukeCaptainAI/norm_quad/blob/main/cosinerule.png?raw=true'
from IPython.display import Image # 
Image(url= sin, width=400, height=400)

In [1]:
# stap 2:
# -------
# De numpy array quad_cog toevoegen aan de oorspronkelijke dataframe 
# 1 op basis van kwadranten info een if statement toevoegen waarbij de cog of wel hoek a gereduceert wordt
# 2 de y waarde middels de sinus van de gerduceerde hoek toevoegen op basis van de coditionele waarde van ['quad_cog']
# 3 de x waarde middels cosinus van ... ...
# 
# ['cog']: a  ['quad_cog']    1['reduced_cog']:b  2['cog_y']                       3['cog_x']
# 53.250000   1               53.250000 a=a       y= np.sin(np.radians(b))         x= np.cos(np.radians(b))
#  
# 111.450001  2               21.450001 a=a-90    y= (np.sin(np.radians(b))*(-1))  x= np.cos(np.radians(b))
#  
# 231.675003  3               51.675003 a=a-180   y= (np.sin(np.radians(b))*(-1))  x= (np.cos(np.radians(b))*(-1))
#  
# 336.000000  4               66.000000 a=a-270   y= np.sin(np.radians(b))         x= (np.cos(np.radians(b))*(-1))

array([124.        ,  25.90000153,  16.59999847,  43.        ,
         6.        ,  80.6000061 ,  21.        , 114.        ,
        13.90000153,  15.39999962,  16.59999847,  88.29998779,
        61.5       ,  43.        ,   4.        , 124.6000061 ,
        26.80000305,  47.1000061 ,  42.        ,  42.        ,
        21.30000305,   0.        ,  24.19999695,  35.        ,
        16.59999847, 118.29998779,  24.5       ,   9.30000019,
        85.        ,  41.        , 113.        ,  21.59999847,
        23.40000153,  79.        ,  30.80000305,  50.5       ,
        87.70001221,  22.09999847,  73.        ,  66.3999939 ,
        15.19999981,  22.69999695, 111.        ,  16.59999847,
        67.        , 125.20001221,  43.        ,  21.69999695,
       110.        ,  41.        ,  24.40000153,  22.        ,
       118.6000061 ,  27.        ,  60.        , 156.        ,
        54.        , 111.        ,  24.        ,  50.20000076,
        51.        ,  21.09999847,  33.69999695,  16.59